# 🔭 RSG Lensing Inversion - Online Demo
**Run all cells → Get shareable link!**

In [ ]:
!pip install -q gradio numpy matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
from dataclasses import dataclass
from typing import Tuple

@dataclass
class RingFit:
    center_x: float
    center_y: float
    radius: float
    rms_residual: float
    radial_residuals: np.ndarray
    azimuthal_angles: np.ndarray
    m2_amplitude: float
    m2_phase: float
    m4_amplitude: float
    m4_phase: float
    perturbation_type: str

def fit_ring(positions: np.ndarray) -> RingFit:
    cx, cy = np.mean(positions, axis=0)
    dx, dy = positions[:, 0] - cx, positions[:, 1] - cy
    radii = np.sqrt(dx**2 + dy**2)
    R = np.mean(radii)
    residuals = radii - R
    phi = np.arctan2(dy, dx)
    # Harmonic analysis
    c2 = 2*np.mean(residuals * np.cos(2*phi))
    s2 = 2*np.mean(residuals * np.sin(2*phi))
    m2_amp = np.sqrt(c2**2 + s2**2)
    m2_phase = np.arctan2(s2, c2) / 2
    c4 = 2*np.mean(residuals * np.cos(4*phi))
    s4 = 2*np.mean(residuals * np.sin(4*phi))
    m4_amp = np.sqrt(c4**2 + s4**2)
    m4_phase = np.arctan2(s4, c4) / 4
    # Perturbation type
    thresh = 0.02 * R
    if m2_amp > thresh and m4_amp > thresh:
        ptype = 'shear + m4'
    elif m2_amp > thresh:
        ptype = 'shear (m=2)'
    elif m4_amp > thresh:
        ptype = 'm=4 only'
    else:
        ptype = 'circular'
    return RingFit(cx, cy, R, np.std(residuals), residuals, phi, m2_amp, m2_phase, m4_amp, m4_phase, ptype)

def classify_morphology(positions: np.ndarray) -> Tuple[str, float, str]:
    n = len(positions)
    if n == 2:
        return 'DOUBLE', 0.9, 'Two-image system'
    if n == 4:
        return 'QUAD', 0.95, 'Einstein Cross (4 images)'
    ring = fit_ring(positions)
    scatter = ring.rms_residual / ring.radius if ring.radius > 0 else 1
    # Azimuthal coverage
    phi_sorted = np.sort(ring.azimuthal_angles)
    gaps = np.diff(np.concatenate([phi_sorted, [phi_sorted[0] + 2*np.pi]]))
    coverage = 1 - np.max(gaps) / (2*np.pi)
    if n >= 10 and scatter < 0.15 and coverage > 0.6:
        return 'RING', min(0.99, 0.7 + coverage*0.3), f'Einstein Ring (scatter={scatter:.2f})'
    elif coverage < 0.5:
        return 'ARC', 0.8, f'Partial arc (coverage={coverage:.1%})'
    else:
        return 'RING', 0.6, f'Ring-like (n={n}, scatter={scatter:.2f})'

print('✅ Functions loaded!')

In [ ]:
def analyze(text):
    try:
        lines = [l.strip() for l in text.strip().split('\n') if l.strip()]
        pos = np.array([[float(x) for x in l.replace(',', ' ').split()[:2]] for l in lines])
        if len(pos) < 2:
            return 'Need >= 2 points', None
        morph, conf, note = classify_morphology(pos)
        ring = fit_ring(pos)
        report = f'''# Analysis Result\n\n
## Morphology: **{morph}** ({conf:.0%})\n{note}\n\n
## Ring Fit\n- Center: ({ring.center_x:.4f}, {ring.center_y:.4f})\n- Radius: {ring.radius:.4f}\n- RMS: {ring.rms_residual:.4f}\n\n
## Harmonics\n- m=2: {ring.m2_amplitude:.4f}\n- m=4: {ring.m4_amplitude:.4f}\n- Type: {ring.perturbation_type}'''
        # Plot
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        ax1, ax2 = axes
        ax1.scatter(pos[:, 0], pos[:, 1], c='red', s=80, zorder=5)
        t = np.linspace(0, 2*np.pi, 100)
        ax1.plot(ring.center_x + ring.radius*np.cos(t), ring.center_y + ring.radius*np.sin(t), 'b--', alpha=0.6)
        ax1.scatter([ring.center_x], [ring.center_y], c='blue', marker='+', s=100)
        ax1.set_aspect('equal')
        ax1.set_title(f'{morph} ({conf:.0%})')
        ax1.grid(True, alpha=0.3)
        # Residual plot
        idx = np.argsort(ring.azimuthal_angles)
        ax2.scatter(np.degrees(ring.azimuthal_angles[idx]), ring.radial_residuals[idx], c='blue', s=40)
        ax2.axhline(0, color='gray', ls='--')
        phi_m = np.linspace(-np.pi, np.pi, 100)
        ax2.plot(np.degrees(phi_m), ring.m2_amplitude*np.cos(2*phi_m - 2*ring.m2_phase), 'g-', alpha=0.7, label=f'm=2: {ring.m2_amplitude:.3f}')
        ax2.plot(np.degrees(phi_m), ring.m4_amplitude*np.cos(4*phi_m - 4*ring.m4_phase), 'orange', alpha=0.7, label=f'm=4: {ring.m4_amplitude:.3f}')
        ax2.set_xlabel('Angle (deg)')
        ax2.set_ylabel('Residual')
        ax2.set_title('Radial Residual vs Angle')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        plt.tight_layout()
        return report, fig
    except Exception as e:
        return f'Error: {e}', None

def generate_synthetic(stype, n, noise, c2, c4):
    n = int(n)
    if stype == 'Quad':
        phi = np.array([0.3, 1.8, 3.5, 5.2])
    else:
        phi = np.linspace(0, 2*np.pi, n, endpoint=False)
    r = 1.0 + c2*np.cos(2*phi) + c4*np.cos(4*phi)
    pos = np.column_stack([r*np.cos(phi), r*np.sin(phi)])
    pos += np.random.normal(0, noise, pos.shape)
    return '\n'.join([f'{p[0]:.4f}, {p[1]:.4f}' for p in pos])

EXAMPLE = '''0.740, 0.565
-0.635, 0.470
-0.480, -0.755
0.870, -0.195'''

with gr.Blocks(title='RSG Lensing') as demo:
    gr.Markdown('# 🔭 RSG Lensing Inversion\nAnalyze Einstein Rings & Crosses')
    with gr.Tabs():
        with gr.Tab('Analyze'):
            with gr.Row():
                inp = gr.Textbox(label='Positions (x, y per line)', lines=8, value=EXAMPLE)
                out_md = gr.Markdown()
            out_plot = gr.Plot()
            gr.Button('Analyze', variant='primary').click(analyze, inp, [out_md, out_plot])
        with gr.Tab('Generate'):
            stype = gr.Dropdown(['Ring', 'Quad'], value='Ring', label='Type')
            n = gr.Slider(4, 50, 20, step=1, label='Points')
            noise = gr.Slider(0, 0.1, 0.01, label='Noise')
            c2 = gr.Slider(0, 0.3, 0, label='m=2')
            c4 = gr.Slider(0, 0.2, 0, label='m=4')
            out_gen = gr.Textbox(label='Generated', lines=8)
            gr.Button('Generate').click(generate_synthetic, [stype, n, noise, c2, c4], out_gen)

demo.launch(share=True)